# Adventure of code 2020

Adventure of code [2020](https://adventofcode.com/2020) in a monolitic notebook.

In [1]:
from math import prod
from collections import namedtuple, Counter
from itertools import combinations
from string import ascii_lowercase, digits
from typing import Tuple

In [2]:
def read_input(path, parse_int=False):
    with open(path) as f:
        for _, line in enumerate(f):

            line = line.strip()
            if parse_int:
                line = int(line)

            yield line

## Day 1

Find the sum equal to the target value.

In [3]:
data = list(read_input("inputs/input_01.txt", parse_int=True))
target = 2020

for title, count in zip(["a", "b"], [2, 3]):
    result = [
        prod(items) for items in combinations(data, count) if sum(items) == target
    ][0]
    print(f"{title} = {result}")

a = 1019904
b = 176647680


## Day 2

Check if a policy (an str) is valid for a given condition.

In [4]:
Policy = namedtuple("policy", ["number_0", "number_1", "letter", "policy"])


def parse_policy(raw_data):
    for raw_item in raw_data:
        rest, policy_str = raw_item.split(":")
        rest, letter = rest.split(" ")
        number_0, number_1 = tuple(map(int, rest.split("-")))
        policy_str = policy_str.strip()

        yield Policy(number_0, number_1, letter, policy_str)


def check_policy_1(policy: Policy) -> bool:
    """Check if the policy count for the 'letter' is inside de numbers range."""
    count = Counter(policy.policy)
    count_letter = count.get(policy.letter, 0)

    if policy.number_0 <= count_letter <= policy.number_1:
        return True
    else:
        return False


def check_policy_2(policy: Policy) -> bool:
    """Check if there is the correct 'letter' in the numbers positions.
    Positions start counting from 1, so we sustract 1."""
    return (
        sum(
            policy.policy[i - 1] == policy.letter
            for i in [policy.number_0, policy.number_1]
        )
        == 1
    )


raw_data = read_input("inputs/input_02.txt")
result = sum(check_policy_1(policy) for policy in parse_policy(raw_data))
print(f"a = {result}")

raw_data = read_input("inputs/input_02.txt")
result = sum(check_policy_2(policy) for policy in parse_policy(raw_data))
print(f"b = {result}")

a = 506
b = 443


## Day 3

## Day 4
Check if a passport (a dict) is valid

In [5]:
valid_keys = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid", "cid"]


def parse_passports(raw_data):
    passport = {}
    for line in raw_data:
        if not line:
            yield passport
            passport = {}

        else:
            for raw_field in line.split():
                k, v = raw_field.split(":")
                passport[k] = v
    # return the last one
    yield passport


def check_passport_1(passport: dict) -> bool:
    """Check if the passport have all the keys (the last one is optional)"""
    return all(k in passport for k in valid_keys[:-1])


def check_generic_field(raw_field: str, low_value: int, upper_value: int) -> bool:
    try:
        field = int(raw_field)
    except ValueError:
        return False

    if len(raw_field) != 4:
        return False

    if not low_value <= field <= upper_value:
        return False

    return True


def check_hgt_field(raw_field: str) -> bool:
    try:
        field = int(raw_field[:-2])
    except ValueError:
        return False

    field_type = raw_field[-2:]

    if field_type not in ["cm", "in"]:
        return False
    elif field_type == "cm":
        low_value = 150
        upper_value = 193
    elif field_type == "in":
        low_value = 59
        upper_value = 76

    if not low_value <= field <= upper_value:
        return False

    return True


def check_hcl_field(raw_field: str) -> bool:
    if raw_field[0] != "#":
        return False

    for char in raw_field[1:]:
        if char not in digits + ascii_lowercase[:6]:
            return False
    return True


def check_ecl_field(raw_field: str) -> bool:
    return raw_field in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]


def check_pid_field(raw_field: str) -> bool:
    if len(raw_field) != 9:
        return False

    return raw_field.isnumeric()


def check_passport_2(passport: dict) -> bool:
    """Check that each passport field is valid."""
    if not all(k in passport for k in valid_keys[:-1]):
        return False

    conditions = [
        check_generic_field(passport["byr"], 1920, 2002),
        check_generic_field(passport["iyr"], 2010, 2020),
        check_generic_field(passport["eyr"], 2020, 2030),
        check_hgt_field(passport["hgt"]),
        check_hcl_field(passport["hcl"]),
        check_ecl_field(passport["ecl"]),
        check_pid_field(passport["pid"]),
    ]

    return all(conditions)


raw_data = read_input("inputs/input_04.txt")
result = sum(map(check_passport_1, parse_passports(raw_data)))
print(f"a = {result}")

raw_data = read_input("inputs/input_04.txt")
result = sum(map(check_passport_2, parse_passports(raw_data)))
print(f"b = {result}")

a = 219
b = 127


## Day 5

In [6]:
def parse_seat(seat: str) -> Tuple[int, int]:
    row = int(seat[:-3].replace("F", "0").replace("B", "1"), 2)
    column = int(seat[-3:].replace("L", "0").replace("R", "1"), 2)
    return row, column


def compute_seat_id(row: int, column: int) -> int:
    return row * 8 + column


def find_missing_seat(all_seats) -> int:
    seats_sorted = sorted(all_seats)
    current_seat = seats_sorted[0]
    for seat in seats_sorted[1:]:
        if seat - current_seat != 1:
            return current_seat + 1  # or "seat - 1"

        current_seat = seat


raw_data = read_input("inputs/input_05.txt")
result = max(
    compute_seat_id(row, column) for row, column in (map(parse_seat, raw_data))
)
print(f"a = {result}")

raw_data = read_input("inputs/input_05.txt")
result = find_missing_seat(
    compute_seat_id(row, column) for row, column in (map(parse_seat, raw_data))
)
print(f"b = {result}")

a = 959
b = 527


## Day 6

## Day 7

## Day 8

In [7]:
Instruction = namedtuple("Instruction", ["operation", "value"])


def parse_instruction(raw_instruction: str) -> Instruction:
    operation, value = raw_instruction.split()
    return Instruction(operation, int(value))


def compute_instruction(instruction, acc, pos):
    if instruction.operation == "acc":
        acc += instruction.value
        pos += 1
    elif instruction.operation == "jmp":
        pos += instruction.value
    elif instruction.operation == "nop":
        pos += 1

    return acc, pos


def run_program(all_instructions):
    """Compute the instructions.
    Return:
        acc: the final acc value
        end_type: False if is an early stop (hit an know instruction), else True
    """
    acc = 0
    pos = 0

    know_pos = set()

    while pos not in know_pos:
        know_pos.add(pos)
        instruction = all_instructions[pos]
        acc, pos = compute_instruction(instruction, acc, pos)
        if pos == len(all_instructions):
            return acc, True

    return acc, False


def find_invalid_instruction(all_instructions):
    """Change one instruction at time and check if program finish with no problems."""
    for i, instruction in enumerate(all_instructions):
        if instruction.operation == "acc":
            continue
        elif instruction.operation == "jmp":
            new_operator = "nop"
        elif instruction.operation == "nop":
            new_operator = "jmp"

        all_instructions_copy = all_instructions.copy()
        all_instructions_copy[i] = Instruction(new_operator, instruction.value)

        acc, end_ok = run_program(all_instructions_copy)
        if end_ok:
            return acc


raw_data = read_input("inputs/input_08.txt")
all_instructions = list(map(parse_instruction, raw_data))
result = run_program(all_instructions)[0]
print(f"a = {result}")

result = find_invalid_instruction(all_instructions)
print(f"b = {result}")

a = 1200
b = 1023
